# Visualizando a malha de serviços com Kiali

Discutiremos sobre muitas ferramentas durante este curso, mas o [Kiali](https://kiali.io/) é a a ferramenta que ajudará você a entender e administrar sua malha de serviços.

Até a versão 1.5, o Kiali e outras ferramentas faziam parte da distribuição do Istio, mas desde a versão 1.6 essas ferramentas devem ser instaladas.

Convenientemente o download do Istio, que fizemos na primeira parte, contém essas ferramentas.

Mas antes de instalar o Kiali, vamos verificar nosso acesso ao cluster:

In [1]:
# Configurando o arquivo de credenciais e acesso ao cluster
export KUBECONFIG=~/.kube/config
# Testando o acesso
kubectl get nodes

NAME             STATUS   ROLES    AGE     VERSION
docker-desktop   Ready    master   7d15h   v1.18.8


Ok, estamos prontos para continuar, mas se tiver algum problema, consulte "Acessando o cluster" na parte 1.

### Instalando o kiali e dependências no cluster

Para instalar o Kiali iremos aplicar o arquivo [kiali.yaml](istio-1.7.3/samples/addons/kiali.yaml), mas antes vamos inspecioná-lo, clique no link para abri-lo.

O arquivo kiali é composto de alguns recursos, são eles:

* CDR
* ServiceAccount
* ConfigMap
* ClusterRole
* ClusterRoleBinding
* Deployment
* Service
* MonitoringDashboard

Com exceção do `MonitoringDashboard`, todos os demais recursos são do kubernetes e o CRD (Custom Resource Definition) é uma forma de criar novos recursos no kubernetes, neste caso, ele define o `MonitoringDashboard`. Mais sobre kuberntes você pode obter no nosso curso [Kubernetes avançado para iniciantes](TODO), onde abordamos esses temas e muitos outros.

Vamos instalar e conhecer o kiali antes de nos aprofundar neste CDR.

Bug versão 1.7.0 a 1.7.3 - [Istio 1.7.1 unable to install Kiali addon #27417](https://github.com/istio/istio/issues/27417)

Necessário aplicar o CRD antes do restante dos recursos.

In [5]:
cat <<EOF | kubectl apply -f -
apiVersion: apiextensions.k8s.io/v1beta1
kind: CustomResourceDefinition
metadata:
  name: monitoringdashboards.monitoring.kiali.io
spec:
  group: monitoring.kiali.io
  names:
    kind: MonitoringDashboard
    listKind: MonitoringDashboardList
    plural: monitoringdashboards
    singular: monitoringdashboard
  scope: Namespaced
  versions:
  - name: v1alpha1
    served: true
    storage: true
EOF

customresourcedefinition.apiextensions.k8s.io/monitoringdashboards.monitoring.kiali.io created


In [6]:
kubectl apply -f istio-1.7.3/samples/addons/kiali.yaml

customresourcedefinition.apiextensions.k8s.io/monitoringdashboards.monitoring.kiali.io unchanged
serviceaccount/kiali created
configmap/kiali created
clusterrole.rbac.authorization.k8s.io/kiali-viewer created
clusterrole.rbac.authorization.k8s.io/kiali created
clusterrolebinding.rbac.authorization.k8s.io/kiali created
service/kiali created
deployment.apps/kiali created
monitoringdashboard.monitoring.kiali.io/envoy created
monitoringdashboard.monitoring.kiali.io/go created
monitoringdashboard.monitoring.kiali.io/kiali created
monitoringdashboard.monitoring.kiali.io/micrometer-1.0.6-jvm-pool created
monitoringdashboard.monitoring.kiali.io/micrometer-1.0.6-jvm created
monitoringdashboard.monitoring.kiali.io/micrometer-1.1-jvm created
monitoringdashboard.monitoring.kiali.io/microprofile-1.1 created
monitoringdashboard.monitoring.kiali.io/microprofile-x.y created
monitoringdashboard.monitoring.kiali.io/nodejs created
monitoringdashboard.monitoring.kiali.io/quarkus created
monitoringdashboar

E o prometheus, responsável por coletar dados dos _containers_ que serão utilizados pelo kiali.

In [17]:
kubectl apply -f istio-1.7.3/samples/addons/prometheus.yaml

serviceaccount/prometheus unchanged
configmap/prometheus unchanged
clusterrole.rbac.authorization.k8s.io/prometheus unchanged
clusterrolebinding.rbac.authorization.k8s.io/prometheus unchanged
service/prometheus unchanged
deployment.apps/prometheus configured


Vamos verificar o que foi instalado.

In [18]:
kubectl get all -n istio-system

NAME                             READY   STATUS    RESTARTS   AGE
pod/istiod-fb4fbff6b-sx2rm       1/1     Running   1          26h
pod/kiali-6c49c7d566-2cksh       1/1     Running   1          26h
pod/prometheus-9d5676d95-tpsf5   2/2     Running   2          26h

NAME                 TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)                                         AGE
service/istiod       ClusterIP   10.104.19.14    <none>        15010/TCP,15012/TCP,443/TCP,15014/TCP,853/TCP   26h
service/kiali        ClusterIP   10.109.13.75    <none>        20001/TCP,9090/TCP                              26h
service/prometheus   ClusterIP   10.101.93.152   <none>        9090/TCP                                        26h

NAME                         READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/istiod       1/1     1            1           26h
deployment.apps/kiali        1/1     1            1           26h
deployment.apps/prometheus   1/1     1            1           26h

NAME     

O serviço do kiali é do tipo `ClusterIP`, o que significa que não podemos acessá-lo diretamente de fora do cluster, há algumas alternativas, modificar ou criar um serviço do tipo `NodePort` ou `LoadBalancer`, configurar um `Ingress` ou usar o subcomando `port-forward` do `kubectl`.

Porém o `istioctl` oferece um subcomando conveniente para acessar o kiali:

In [12]:
istioctl dashboard

Access to Istio web UIs

Usage:
  istioctl dashboard [flags]
  istioctl dashboard [command]

Aliases:
  dashboard, dash, d

Available Commands:
  controlz    Open ControlZ web UI
  envoy       Open Envoy admin web UI
  grafana     Open Grafana web UI
  jaeger      Open Jaeger web UI
  kiali       Open Kiali web UI
  prometheus  Open Prometheus web UI
  zipkin      Open Zipkin web UI

Flags:
      --address string   Address to listen on. Only accepts IP address or localhost as a value. When localhost is supplied, istioctl will try to bind on both 127.0.0.1 and ::1 and will fail if neither of these address are available to bind. (default "localhost")
  -h, --help             help for dashboard
  -p, --port int         Local port to listen to

Global Flags:
      --context string          The name of the kubeconfig context to use
  -i, --istioNamespace string   Istio system namespace (default "istio-system")
  -c, --kubeconfig string       Kubernetes configuration file
  -n, --namespace s

Vamos acessa-lo:

In [2]:
istioctl dashboard kiali

http://localhost:20001/kiali


E voilá, você está acessando o dashboard do kiali, mas oferece acesso a outros dashboards.

Vamos explorar alguns recursos do kiali.

[![Everything Is AWESOME](https://img.youtube.com/vi/3sV5PKfAlG0/0.jpg)](https://youtu.be/3sV5PKfAlG0 "Kiali Graph Replay via Traffic Routing")

Interrompa a execução do dashboard:
* No jupyter Lab ou notebook: Clique no icone <kbd>◾</kbd> (_Interrup the kernel_ na barra de ferramentas)
* No terminal: tecle <kbd>CTRL</kbd>+<kbd>C</kbd>

Se você assistiu nosso curso de [Kubernetes avançado para iniciantes](TODO) deve imaginar o que o `istioctl`automatizou, foi o comando `kubectl port-forward` e adicionou um comando de ` open` para abrir a página inicial no navegador.

O comando a seguir tem efeito semelhante (sem a parte do navegador)

In [13]:
kubectl port-forward service/kiali 20001:20001 -n istio-system

Forwarding from 127.0.0.1:20001 -> 20001
Forwarding from [::1]:20001 -> 20001
Handling connection for 20001
Handling connection for 20001
Handling connection for 20001
Handling connection for 20001
Handling connection for 20001
Handling connection for 20001


Mesmo resultado, mas não tão elegante. Ficaremos com o `istioctl dashboard <dashboard>` pelo resto do curso.

Interrompa a execução do `port-forward`.

E vamos executar o kiali novamente, mas agora em segundo plano.

In [20]:
istioctl dashboard kiali &
export KIALI_PID=$!

[1] 5786


### Labels

Para kiali, os _labels_ `app` e `version` são utilizados para identificar os serviços e as cargas de trabalho.

Podemos utilizar o mesmo artifício para localizar qualquer recurso no kuberentes:

In [14]:
kubectl describe deploy -l app=simple-app

Name:                   simple-app
Namespace:              default
CreationTimestamp:      Thu, 22 Oct 2020 09:56:59 -0300
Labels:                 app=simple-app
                        version=1.0
Annotations:            deployment.kubernetes.io/revision: 1
Selector:               app=simple-app,version=1.0
Replicas:               1 desired | 1 updated | 1 total | 1 available | 0 unavailable
StrategyType:           RollingUpdate
MinReadySeconds:        0
RollingUpdateStrategy:  25% max unavailable, 25% max surge
Pod Template:
  Labels:  app=simple-app
           version=1.0
  Containers:
   simple-app:
    Image:        nginx:stable
    Port:         80/TCP
    Host Port:    0/TCP
    Environment:  <none>
    Mounts:       <none>
  Volumes:        <none>
Conditions:
  Type           Status  Reason
  ----           ------  ------
  Available      True    MinimumReplicasAvailable
  Progressing    True    NewReplicaSetAvailable
OldReplicaSets:  <none>
NewReplicaSet:   simple-app-6f6cfc85

E no nosso `deployment` adicionamos na seção _Pod Template_ os rótulos que desejamos que sejam adicionados aos _PODs_ que forem criados.

In [15]:
kubectl get pods -l app=simple-app

NAME                          READY   STATUS    RESTARTS   AGE
simple-app-6f6cfc8554-6m6t2   2/2     Running   0          73m


Podemos utilizar multiplos _labels_ para localizar um recurso.

In [16]:
kubectl get all -l app=simple-app -l version=1.0

NAME                              READY   STATUS    RESTARTS   AGE
pod/simple-app-6f6cfc8554-6m6t2   2/2     Running   0          74m

NAME                 TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)   AGE
service/simple-app   ClusterIP   10.109.84.250   <none>        80/TCP    74m

NAME                         READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/simple-app   1/1     1            1           74m

NAME                                    DESIRED   CURRENT   READY   AGE
replicaset.apps/simple-app-6f6cfc8554   1         1         1       74m


O kiali não é uma ferramenta estática, uma das suas forças e exibir a relação entre os recursos em tempo de execução. Na próxima parte iremos explorar uma aplicação com uma malha de serviços.

Por agora podemos para o dashboard do kiali, para isso precisaremos parar o processo.

In [21]:
kill $KIALI_PID

http://localhost:20001/kiali


: 1

## Limpando

Não precisaremos mais da nossa aplicação de teste, vamos exluí-la para liberar recursos do cluster.

Os recursos criados podem ser obtidos passando as mesmas configurações que utilizamos para criá-los.

In [25]:
kubectl get -f exemplos/simple-app

NAME                         READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/simple-app   1/1     1            1           89m

NAME                 TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)   AGE
service/simple-app   ClusterIP   10.109.84.250   <none>        80/TCP    89m


E o mesmo vale para excluí-los.

> O kubectl não solicita confirmação para execução, tome cuidado e revise o comando antes de executá-lo.

In [27]:
kubectl delete -f exemplos/simple-app

deployment.apps "simple-app" deleted
service "simple-app" deleted
